In [95]:
# Region to install edge function. Note edge functions need to be created in us-east-1
# because reasons.
my_region = 'us-east-1'

In [96]:
# SDK setup
import boto3

cloudformation = boto3.client('cloudformation',region_name = my_region)

In [97]:
# Upload the zip and cf template
bucket_name = 'code97068'

In [98]:
%%bash -s "$bucket_name"
make
aws s3 cp edge.yml s3://$1
aws s3 cp router.zip s3://$1

rm -f router.zip
zip router.zip router.js
  adding: router.js (deflated 47%)
upload: ./edge.yml to s3://code97068/edge.yml                 
upload: ./router.zip to s3://code97068/router.zip                 


In [99]:
def form_s3_url_prefix(region):
    prefix = ''
    if region == 'us-east-1':
        prefix = 'https://s3.amazonaws.com'
    else:
        prefix = 'https://s3-' + region + '.amazonaws.com'
    return prefix

In [100]:
edge_fn_stack = 'bgrouter1'
bucket_base = form_s3_url_prefix(my_region) + '/' + bucket_name
print(bucket_base)

https://s3.amazonaws.com/code97068


In [101]:
response = cloudformation.create_stack(
    StackName=edge_fn_stack,
    TemplateURL= bucket_base + '/edge.yml',
    Parameters=[
        {
            'ParameterKey': 'CodeBucketName',
            'ParameterValue':bucket_name
        }
    ],
    Capabilities=[
        'CAPABILITY_IAM'
    ]
)

print(response)

{'StackId': 'arn:aws:cloudformation:us-east-1:427848627088:stack/bgrouter1/7949add0-68f8-11e8-a778-500c5240582a', 'ResponseMetadata': {'RequestId': '79428251-68f8-11e8-8109-45c2f7c50e78', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '79428251-68f8-11e8-8109-45c2f7c50e78', 'content-type': 'text/xml', 'content-length': '379', 'date': 'Tue, 05 Jun 2018 19:41:42 GMT'}, 'RetryAttempts': 0}}


In [102]:
print('waiting on create of {}'.format(edge_fn_stack))
waiter = cloudformation.get_waiter('stack_create_complete')
waiter.wait(
    StackName=edge_fn_stack
)

print('stack created')

waiting on create of bgrouter1
stack created


In [103]:
#Get the arn of the lambda function created above. We will need this to 
# set the edge function in our cloud front distribution.
response = cloudformation.describe_stacks(
    StackName=edge_fn_stack
)

print(response)

outputs = response['Stacks'][0]['Outputs']
lambda_arn = [d for d in outputs if d['OutputKey'] == 'LambdaArn'][0]['OutputValue']
print(lambda_arn)

{'Stacks': [{'StackId': 'arn:aws:cloudformation:us-east-1:427848627088:stack/bgrouter1/7949add0-68f8-11e8-a778-500c5240582a', 'StackName': 'bgrouter1', 'Description': 'Create an edge function that can be associated with a cloud front distribution. This is separate from the cloud front distro at the moment as the bucket name needs to be hard coded as it cannot be injected via environment vars due to edge function restrictions.\n', 'Parameters': [{'ParameterKey': 'CodeBucketName', 'ParameterValue': 'code97068'}], 'CreationTime': datetime.datetime(2018, 6, 5, 19, 41, 43, 155000, tzinfo=tzutc()), 'RollbackConfiguration': {}, 'StackStatus': 'CREATE_COMPLETE', 'DisableRollback': False, 'NotificationARNs': [], 'Capabilities': ['CAPABILITY_IAM'], 'Outputs': [{'OutputKey': 'LambdaArn', 'OutputValue': 'arn:aws:lambda:us-east-1:427848627088:function:bgrouter1-EdgeProto-1PFL1SBUXZIO2'}], 'Tags': [], 'EnableTerminationProtection': False}], 'ResponseMetadata': {'RequestId': 'b3c6ac4f-68f8-11e8-b06f-

In [104]:
# Now we need to switch to region where the site distro stack was created
cdn_region = 'us-east-1'
cloudfront = boto3.client('cloudfront',region_name = cdn_region)
cloudformation = boto3.client('cloudformation',region_name = cdn_region)
cloudfront_stack_name = 'bg1'

In [105]:
response = cloudformation.describe_stack_resource(
    StackName=cloudfront_stack_name,
    LogicalResourceId='MyDistro'
)

distro_id = response['StackResourceDetail']['PhysicalResourceId']

print(distro_id)


E2W4Z3UU56NZ6Q


In [107]:
response = cloudfront.get_distribution_config(
    Id=distro_id
)

etag = response['ETag']
config = response['DistributionConfig']

print(config)

{'CallerReference': 'df1dac54-b4cd-48c5-874a-be4e3ae463f4', 'Aliases': {'Quantity': 0}, 'DefaultRootObject': '', 'Origins': {'Quantity': 3, 'Items': [{'Id': 'default', 'DomainName': 'bg1-defaultbucket-13crklj51b5uv.s3.amazonaws.com', 'OriginPath': '', 'CustomHeaders': {'Quantity': 0}, 'S3OriginConfig': {'OriginAccessIdentity': 'origin-access-identity/cloudfront/E1H23UGIMIGJXU'}}, {'Id': 'green', 'DomainName': 'bg1-greenbucket-1uxs4t2hxe1jr.s3.amazonaws.com', 'OriginPath': '', 'CustomHeaders': {'Quantity': 0}, 'S3OriginConfig': {'OriginAccessIdentity': 'origin-access-identity/cloudfront/E1H23UGIMIGJXU'}}, {'Id': 'blue', 'DomainName': 'bg1-bluebucket-1oiwc3xnxyqzc.s3.amazonaws.com', 'OriginPath': '', 'CustomHeaders': {'Quantity': 0}, 'S3OriginConfig': {'OriginAccessIdentity': 'origin-access-identity/cloudfront/E1H23UGIMIGJXU'}}]}, 'DefaultCacheBehavior': {'TargetOriginId': 'default', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'Qu

In [108]:
print(config['DefaultCacheBehavior'])
for key in config['DefaultCacheBehavior']:
    print(key)

{'TargetOriginId': 'default', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'QueryStringCacheKeys': {'Quantity': 0}}, 'TrustedSigners': {'Enabled': False, 'Quantity': 0}, 'ViewerProtocolPolicy': 'redirect-to-https', 'MinTTL': 0, 'AllowedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET'], 'CachedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET']}}, 'SmoothStreaming': False, 'DefaultTTL': 300, 'MaxTTL': 600, 'Compress': False, 'LambdaFunctionAssociations': {'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:bgrouter1-EdgeProto-LFS0ERENFNYU:1', 'EventType': 'origin-request'}]}, 'FieldLevelEncryptionId': ''}
TargetOriginId
ForwardedValues
TrustedSigners
ViewerProtocolPolicy
MinTTL
AllowedMethods
SmoothStreaming
DefaultTTL
MaxTTL
Compress
LambdaFunctionAssociations
FieldLevelEncryptionId


In [109]:
lambda_version = '1'
versioned_lambda_arn = lambda_arn + ':' + lambda_version
lambdaAssociation = {'Quantity': 1, 'Items':[{'LambdaFunctionARN':versioned_lambda_arn, 'EventType':'origin-request'}]}
print(lambdaAssociation)

{'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:bgrouter1-EdgeProto-1PFL1SBUXZIO2:1', 'EventType': 'origin-request'}]}


In [110]:
config['DefaultCacheBehavior']['LambdaFunctionAssociations'] = lambdaAssociation

In [111]:
print(config['DefaultCacheBehavior'])

{'TargetOriginId': 'default', 'ForwardedValues': {'QueryString': False, 'Cookies': {'Forward': 'none'}, 'Headers': {'Quantity': 0}, 'QueryStringCacheKeys': {'Quantity': 0}}, 'TrustedSigners': {'Enabled': False, 'Quantity': 0}, 'ViewerProtocolPolicy': 'redirect-to-https', 'MinTTL': 0, 'AllowedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET'], 'CachedMethods': {'Quantity': 2, 'Items': ['HEAD', 'GET']}}, 'SmoothStreaming': False, 'DefaultTTL': 300, 'MaxTTL': 600, 'Compress': False, 'LambdaFunctionAssociations': {'Quantity': 1, 'Items': [{'LambdaFunctionARN': 'arn:aws:lambda:us-east-1:427848627088:function:bgrouter1-EdgeProto-1PFL1SBUXZIO2:1', 'EventType': 'origin-request'}]}, 'FieldLevelEncryptionId': ''}


In [112]:
update_response = cloudfront.update_distribution(
    DistributionConfig=config,
    Id=distro_id,
    IfMatch=etag
)

print(update_response)

{'ResponseMetadata': {'RequestId': 'bb203c27-68f8-11e8-9a65-5398ac69d75a', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'bb203c27-68f8-11e8-9a65-5398ac69d75a', 'etag': 'E2J4LDHEGMUQXT', 'content-type': 'text/xml', 'content-length': '3580', 'vary': 'Accept-Encoding', 'date': 'Tue, 05 Jun 2018 19:43:33 GMT'}, 'RetryAttempts': 0}, 'ETag': 'E2J4LDHEGMUQXT', 'Distribution': {'Id': 'E2W4Z3UU56NZ6Q', 'ARN': 'arn:aws:cloudfront::427848627088:distribution/E2W4Z3UU56NZ6Q', 'Status': 'InProgress', 'LastModifiedTime': datetime.datetime(2018, 6, 5, 19, 43, 33, 845000, tzinfo=tzutc()), 'InProgressInvalidationBatches': 0, 'DomainName': 'd3q95kwcioahms.cloudfront.net', 'ActiveTrustedSigners': {'Enabled': False, 'Quantity': 0}, 'DistributionConfig': {'CallerReference': 'df1dac54-b4cd-48c5-874a-be4e3ae463f4', 'Aliases': {'Quantity': 0}, 'DefaultRootObject': '', 'Origins': {'Quantity': 3, 'Items': [{'Id': 'default', 'DomainName': 'bg1-defaultbucket-13crklj51b5uv.s3.amazonaws.com', 'OriginPa